# Suburb Venue Profiles and the Sydney Housing Market

### Parse CSV file of Sydney Suburb Names into Pandas Dataframe

In [72]:
# List of subrubs from "http://www.walksydneystreets.net/suburbssydneyall.htm"

In [39]:
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3
import geopy

# The CSV file is uploaded to IBM Watson, which is accessed using my credentials. This is why the next cell is hidden.

In [40]:
# The code was removed by Watson Studio for sharing.

In [41]:
suburbs = df_data_0.T
suburbs.columns = ['Suburb Name']
suburbs.head()

,Suburb Name
0,Abbotsbury
1,Abbotsford
2,Acacia Gardens
3,Agnes Banks
4,Airds


In [42]:
suburbs["Address"] = suburbs["Suburb Name"] + ", New South Wales, Australia"
suburbs.head()

(669, 2)

### Get coordinates of each suburb using geocoder

In [43]:
locator = geopy.Nominatim(user_agent="chickentar@hotmail.com")

In [ ]:
from geopy.extra.rate_limiter import RateLimiter

# 1 - conveneint function to delay between geocoding calls
geocode = RateLimiter(locator.geocode, min_delay_seconds=2)
# 2- - create location column
suburbs['location'] = suburbs['Address'].apply(geocode)
# 3 - create longitude, laatitude and altitude from location column (returns tuple)
suburbs['point'] = suburbs['location'].apply(lambda loc: tuple(loc.point) if loc else None)
# 4 - split point column into latitude, longitude and altitude columns
suburbs[['latitude', 'longitude', 'altitude']] = pd.DataFrame(suburbs['point'].tolist(), index=suburbs.index)

suburbs.head()

RateLimiter caught an error, retrying (0/2 tries). Called with (*('Alexandria, New South Wales, Australia',), **{}).
Traceback (most recent call last):
  File "/opt/conda/envs/Python36/lib/python3.6/urllib/request.py", line 1318, in do_open
    encode_chunked=req.has_header('Transfer-encoding'))
  File "/opt/conda/envs/Python36/lib/python3.6/http/client.py", line 1239, in request
    self._send_request(method, url, body, headers, encode_chunked)
  File "/opt/conda/envs/Python36/lib/python3.6/http/client.py", line 1285, in _send_request
    self.endheaders(body, encode_chunked=encode_chunked)
  File "/opt/conda/envs/Python36/lib/python3.6/http/client.py", line 1234, in endheaders
    self._send_output(message_body, encode_chunked=encode_chunked)
  File "/opt/conda/envs/Python36/lib/python3.6/http/client.py", line 1026, in _send_output
    self.send(msg)
  File "/opt/conda/envs/Python36/lib/python3.6/http/client.py", line 964, in send
    self.connect()
  File "/opt/conda/envs/Python36/l